<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/Porto%20Seguro%E2%80%99s%20Safe%20Driver%20Prediction/Porto_Seguro%E2%80%99s_Safe_Driver_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packaging


In [0]:
import numpy as np
import pandas as pd
import os
import sys


In [2]:
COLAB=True
if(COLAB):
  from google.colab import drive
  drive.mount('/content/drive/')
else:
  pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [3]:
path = '/content/drive/My Drive/Datasets/Porto Seguro’s Safe Driver Prediction/'
train = pd.read_csv(path+'train.csv')
test = pd.read_csv(path+'test.csv')
print("Train Shape: ", train.shape)
print("Test Shape: ", test.shape)
train_id = train['id']
test_id = test['id']
y = train['target']
train = train.drop(columns=['id', 'target'])
test = test.drop(columns=['id'])
df = pd.concat([train, test])
print('Df Shape: ', df.shape)
# # Info
print(df.info())

Train Shape:  (595212, 59)
Test Shape:  (892816, 58)
Df Shape:  (1488028, 57)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1488028 entries, 0 to 892815
Data columns (total 57 columns):
ps_ind_01         1488028 non-null int64
ps_ind_02_cat     1488028 non-null int64
ps_ind_03         1488028 non-null int64
ps_ind_04_cat     1488028 non-null int64
ps_ind_05_cat     1488028 non-null int64
ps_ind_06_bin     1488028 non-null int64
ps_ind_07_bin     1488028 non-null int64
ps_ind_08_bin     1488028 non-null int64
ps_ind_09_bin     1488028 non-null int64
ps_ind_10_bin     1488028 non-null int64
ps_ind_11_bin     1488028 non-null int64
ps_ind_12_bin     1488028 non-null int64
ps_ind_13_bin     1488028 non-null int64
ps_ind_14         1488028 non-null int64
ps_ind_15         1488028 non-null int64
ps_ind_16_bin     1488028 non-null int64
ps_ind_17_bin     1488028 non-null int64
ps_ind_18_bin     1488028 non-null int64
ps_reg_01         1488028 non-null float64
ps_reg_02         1488028 non

In [4]:
print(df.describe())

          ps_ind_01  ps_ind_02_cat     ps_ind_03  ps_ind_04_cat  \
count  1.488028e+06   1.488028e+06  1.488028e+06   1.488028e+06   
mean   1.901574e+00   1.358745e+00  4.417567e+00   4.171346e-01   
std    1.985417e+00   6.636391e-01  2.700054e+00   4.933963e-01   
min    0.000000e+00  -1.000000e+00  0.000000e+00  -1.000000e+00   
25%    0.000000e+00   1.000000e+00  2.000000e+00   0.000000e+00   
50%    1.000000e+00   1.000000e+00  4.000000e+00   0.000000e+00   
75%    3.000000e+00   2.000000e+00  6.000000e+00   1.000000e+00   
max    7.000000e+00   4.000000e+00  1.100000e+01   1.000000e+00   

       ps_ind_05_cat  ps_ind_06_bin  ps_ind_07_bin  ps_ind_08_bin  \
count   1.488028e+06   1.488028e+06   1.488028e+06   1.488028e+06   
mean    4.069547e-01   3.934442e-01   2.571276e-01   1.637637e-01   
std     1.353300e+00   4.885141e-01   4.370505e-01   3.700612e-01   
min    -1.000000e+00   0.000000e+00   0.000000e+00   0.000000e+00   
25%     0.000000e+00   0.000000e+00   0.000000e+00 

In [5]:
# Check null
def get_missing_features(df):
  missings = pd.DataFrame([], columns=['feature', 'no_recoreds', 'percentage'])
  total_rows = df.shape[0]
  index=0
  for feature in list(df):
    total_nulls = df[feature].isnull().sum()
    if(total_nulls>0):
      missing_perc = total_nulls/total_rows
      missings.loc[index] = [feature, total_nulls, missing_perc]
      index+=1
  missings = missings.sort_values('no_recoreds', ascending=False)
  return missings
df_missings = get_missing_features(train)
print(df_missings)

Empty DataFrame
Columns: [feature, no_recoreds, percentage]
Index: []


In [6]:
print(df.isnull().sum().sort_values(ascending=False).head(5))

ps_calc_20_bin    0
ps_car_07_cat     0
ps_car_05_cat     0
ps_car_04_cat     0
ps_car_03_cat     0
dtype: int64


In [7]:
unique_values = df.nunique().sort_values(ascending=True)

for col in df.columns:
  if(df[col].nunique()<50):
    df = pd.get_dummies(df, columns = [col], prefix=col, drop_first=True)
print(df.shape)
    
  

(1488028, 339)


In [0]:
from sklearn.model_selection import StratifiedKFold

class Create_ensemble(object):
    def __init__(self, n_splits, base_models):
        self.n_splits = n_splits
        self.base_models = base_models

    def predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=random_state).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, clf in enumerate(self.base_models):
            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, valid_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_valid = X[valid_idx]
                y_valid = y[valid_idx]
                
                clf.fit(X_train, y_train)
                valid_pred = clf.predict_proba(X_valid)[:,1]
                S_train[valid_idx, i] = valid_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            
            print( "\nTraining Gini for model {} : {}".format(i, eval_gini(y, S_train[:,i])))
            S_test[:, i] = S_test_i.mean(axis=1)
            
        return S_train, S_test
# from https://www.kaggle.com/mashavasilenko/
# porto-seguro-xgb-modeling-and-parameters-tuning
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini


def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

In [9]:
!pip install lightgbm
from lightgbm import LGBMClassifier
random_state = 6
# LightGBM params
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 700
lgb_params['max_bin'] = 15
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8   
lgb_params['min_child_samples'] = 800
lgb_params['random_state'] = random_state
lgb_params['scale_pos_weight'] = 3

lgb_params2 = {}
lgb_params2['learning_rate'] = 0.02
lgb_params2['n_estimators'] = 900
lgb_params2['max_bin'] = 20
lgb_params2['subsample'] = 0.8
lgb_params2['subsample_freq'] = 10
lgb_params2['colsample_bytree'] = 0.8   
lgb_params2['min_child_samples'] = 600
lgb_params2['random_state'] = random_state
lgb_params2['scale_pos_weight'] = 3

lgb_model = LGBMClassifier(**lgb_params)
lgb_model2 = LGBMClassifier(**lgb_params2)

In [10]:
df_train = df.iloc[:train.shape[0], :]
df_test = df.iloc[train.shape[0]:, :]
print(df_train.shape)
print(train.shape)
print(df_test.shape)
print(test.shape)

lgb_stack = Create_ensemble(n_splits = 5, base_models = [lgb_model, lgb_model2])        
X = df_train
Y = y
T = df_test
lgb_train_pred, lgb_test_pred = lgb_stack.predict(X, Y, T)

(595212, 339)
(595212, 57)
(892816, 339)
(892816, 57)

Training Gini for model 0 : 0.2715522812390839

Training Gini for model 1 : 0.27076584087229827


In [0]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = lgb_test_pred.mean(axis=1)
sub.to_csv('lightgbm_submit_ensemble_features.csv', float_format='%.6f', index=False)

In [19]:
from sklearn.metrics import accuracy_score
print("Train accuracy: ", accuracy_score(y_train, lr.predict(X_train)))

print("Train accuracy: ", accuracy_score(y_val, lr.predict(X_val)))

Train accuracy:  0.9910526610539678
Train accuracy:  0.962719666677867


In [25]:
print(lr.predict_proba(test)[:, 0])

[1.  0.8 0.8 ... 1.  0.9 1. ]


In [0]:
submit = pd.DataFrame()
submit['id'] = test_id
submit['target'] = lr.predict_proba(test)[:, 1]
submit.to_csv('sub.csv', index=False)